In [ ]:
from op_datasets.processing.execute import reader, BatchInput

inputbatch: BatchInput = None
for _inputbatch in reader(
    chain="op", block_spec="105376883:+500", source_spec="goldsky"
):
    inputbatch = _inputbatch
    break

dataframes = inputbatch.dataframes

In [ ]:
dataframes

In [3]:
def my_logic_for_address_model(_dataframes):
    return _dataframes["transactions"]

In [ ]:
my_logic_for_address_model(dataframes)

In [ ]:
import requests

url = "https://radial-falling-darkness.base-mainnet.quiknode.pro/8de4ba12a3442c43576bb06d1f7d97a56ccd52f3/"

payload = {
    "id": 1,
    "jsonrpc": "2.0",
    "method": "eth_getLogs",
    "params": [{"fromBlock": "0x140320f", "toBlock": "latest"}],
}
headers = {"accept": "application/json", "content-type": "application/json"}

response = requests.post(url, json=payload, headers=headers)

print(response.text)

In [ ]:
response

In [ ]:
response.json()